# APRIORI ALGORİTMASI KULLANARAK BİRLİKTELİK KURAL ANALİZİ 

X: 1.ürün Y: 2.ürün N: Toplam Alışveriş

**- Destek (Support) : Ürünlerin Birlikte Geçme Olasılığı**

Support(X, Y) = Freq(X,Y)/N

(X ve Y farklı ürünlerinin birlikte geçme frekansı / Toplam Alışveriş)

Support değerini hesaplama amacımız eşik değeri belirlemek. Veri setine bakıldığında bir çok ürün beraber görülebilir eşik değerinin altında kalan ürünlerde eleme yapacağız.

**- Güven (Confidence) : X’i Alanların Y’yi Alma Olasılığı**

Confidence(X, Y) = Freq(X,Y)/Freq(X)

(X ve Y farklı ürünlerinin birlikte geçme frekansı / X’in gözlenme frekansı)

Analiz yapıldığında ister support değerine göre ister confidence değerine göre eşik değeri belirlenebilir. Eşik değerini gerçek hayat verilerinde çok az alabiliriz.

**- Lift = Support(X,Y)/(Support(X)*Support(Y))**

X ürünü alanların Y ürünü satın alması şu kadar kat artıyor yorumu vardır.

# Data Understanding

In [1]:
#ROOT_DIR = "/content/drive/MyDrive/..."
ROOT_DIR = "https://raw.githubusercontent.com/yavuzKomecoglu/yapay-zeka-egitimi-tubitak-bideb-2237a/main/"
DATASET_PATH = ROOT_DIR + "datasets/apyori/GroceryStoreDataSet.csv"

In [2]:
# .csv formatındaki veri setini okuma işlemi
# MILK,BREAD,BISCUIT olan sutün başlığını names=['products'] ile products'a çevirdik.

import pandas as pd
df = pd.read_csv(DATASET_PATH, names=['products'], header = None)
df.head()

,products
0,"MILK,BREAD,BISCUIT"
1,"BREAD,MILK,BISCUIT,CORNFLAKES"
2,"BREAD,TEA,BOURNVITA"
3,"JAM,MAGGI,BREAD,MILK"
4,"MAGGI,TEA,BISCUIT"


**Yapılacaklar: Veri kümesinin shape'ini kontrol ediniz.**

In [3]:
#Yapılacaklar: Dataframe'in shape'ini kontrol ediniz.
# 20 gözlem ve 1 sütun var.
df.shape

(20, 1)

**Yapılacaklar: Veri kümesinin sütun isimlerini kontrol ediniz.**

In [4]:
# Sutün adı ve tipi.
df.columns

Index(['products'], dtype='object')

**Yapılacaklar: Veri kümesinin her satırdaki değerlerini yazdırınız.**

In [5]:
# Her satırda gözlenen gözlemler
df.values

array([['MILK,BREAD,BISCUIT'],
       ['BREAD,MILK,BISCUIT,CORNFLAKES'],
       ['BREAD,TEA,BOURNVITA'],
       ['JAM,MAGGI,BREAD,MILK'],
       ['MAGGI,TEA,BISCUIT'],
       ['BREAD,TEA,BOURNVITA'],
       ['MAGGI,TEA,CORNFLAKES'],
       ['MAGGI,BREAD,TEA,BISCUIT'],
       ['JAM,MAGGI,BREAD,TEA'],
       ['BREAD,MILK'],
       ['COFFEE,COCK,BISCUIT,CORNFLAKES'],
       ['COFFEE,COCK,BISCUIT,CORNFLAKES'],
       ['COFFEE,SUGER,BOURNVITA'],
       ['BREAD,COFFEE,COCK'],
       ['BREAD,SUGER,BISCUIT'],
       ['COFFEE,SUGER,CORNFLAKES'],
       ['BREAD,SUGER,BOURNVITA'],
       ['BREAD,COFFEE,SUGER'],
       ['BREAD,COFFEE,SUGER'],
       ['TEA,MILK,COFFEE,CORNFLAKES']], dtype=object)

**Yapılacaklar: Her satırdaki gözlem değerlerini virgül ile birleştirerek bir dizi içerisine tanımlayınız.**

Not: Bir satırda, birden fazla eleman(ürün) olabilir. Birlikte olanları bozmadan sadece satırları ayrı bir dizi elemanı olarak eklemelisiniz.

Birleştirme sonucu çıktı aşağıdaki gibi olmalıdır.

```
[['MILK', 'BREAD', 'BISCUIT'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'COCK'],
 ['BREAD', 'SUGER', 'BISCUIT'],
 ['COFFEE', 'SUGER', 'CORNFLAKES'],
 ['BREAD', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]
```

In [6]:
# Her satırda tek bir gözlem birimi varmış gibiydi bunu virgül ile tek tek ayırma işlemi yaptık.
data = list(df["products"].apply(lambda x:x.split(',')))
data 

[['MILK', 'BREAD', 'BISCUIT'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'COCK'],
 ['BREAD', 'SUGER', 'BISCUIT'],
 ['COFFEE', 'SUGER', 'CORNFLAKES'],
 ['BREAD', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

# Data Preprocessing

In [7]:
from mlxtend.preprocessing import TransactionEncoder

**Yapılacaklar: One-Hot encodinge benzer şekilde TransactionEncoder() ile fit() ve transform() yaparak ürün isimlerinin sütun adlarına yazıldığı ve her bir satırın sadece içerdiği ürünlerde True diğerlerinde False yazdığı yeni bir DataFrame'e dönüştürünüz.**

**Link:** http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/

**Çıktı:**

| BISCUIT | BOURNVITA | BREAD | COCK | COFFEE | CORNFLAKES | JAM | MAGGI | MILK | SUGER | TEA |
|----|----|----|----|----|----|----|----|----|----|----|
|True|False|True|False|False|False|False|False|True|False|False|
|True|False|True|False|False|True|False|False|True|False|False|
.
.
.


 	 	 	 	 	 	 	 	 	


In [8]:
#Veri kümesini istenilen True-False array'ine çevirdik.

te = TransactionEncoder()
te_data = te.fit(data).transform(data)
df = pd.DataFrame(te_data, columns=te.columns_)
df

,BISCUIT,BOURNVITA,BREAD,COCK,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGER,TEA
0,True,False,True,False,False,False,False,False,True,False,False
1,True,False,True,False,False,True,False,False,True,False,False
2,False,True,True,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,True,True,True,False,False
4,True,False,False,False,False,False,False,True,False,False,True
5,False,True,True,False,False,False,False,False,False,False,True
6,False,False,False,False,False,True,False,True,False,False,True
7,True,False,True,False,False,False,False,True,False,False,True
8,False,False,True,False,False,False,True,True,False,False,True
9,False,False,True,False,False,False,False,False,True,False,False


# Data Association Rules

In [ ]:
from mlxtend.frequent_patterns import apriori

**Yapılacaklar: apriori() fonksiyonunu kullanarak veri kümesindeki ürünlerine support değerleri ile birlikte itemsets'i hesaplattırıp yeni bir DataFrame olarak yazdırınız.**

**Link:** http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

In [ ]:
# Apriori Fonksiyonunu deneyelim.

# df = True-False array ya da One Hot Encoding ile dönüştürülmüş dataframe
# min_support = Tüm kombinasyonların support değerini istemiyorum bu yüzden belirli bir eşik değerini getir.
# use_colnames = True, sutün isimlerini göster.

df1 = apriori(df, min_support=0.02, use_colnames=True)
df1

,support,itemsets
0,0.35,(BISCUIT)
1,0.20,(BOURNVITA)
2,0.65,(BREAD)
3,0.15,(COCK)
4,0.40,(COFFEE)
...,...,...
78,0.05,"(TEA, MAGGI, BISCUIT, BREAD)"
79,0.10,"(COFFEE, COCK, CORNFLAKES, BISCUIT)"
80,0.05,"(MAGGI, MILK, BREAD, JAM)"
81,0.05,"(TEA, MAGGI, BREAD, JAM)"


```
BISCUIT tüm alışverişlerin % 35'inde, 
BREAD tüm alışverişlerin %65'inde veya 
TEA, MAGGI, BREAD, BISCUIT tüm satışların % 5'inde beraber yorumları yapılır.
```

**Yapılacaklar: apriori hesabı yapılan ve yeni tanımlanan dataframe'i support değerlerine göre büyükten küçüğe sıralayınızç **

In [ ]:
#Alışverişlerde en çok alınan ürünleri yorumlayabilmek için veriyi büyükten küçüğe sıralayabiliriz.

df1.sort_values(by="support", ascending=False)

,support,itemsets
2,0.65,(BREAD)
4,0.40,(COFFEE)
0,0.35,(BISCUIT)
10,0.35,(TEA)
5,0.30,(CORNFLAKES)
...,...,...
55,0.05,"(MILK, CORNFLAKES, BISCUIT)"
57,0.05,"(SUGER, BREAD, BOURNVITA)"
17,0.05,"(BISCUIT, SUGER)"
37,0.05,"(CORNFLAKES, MAGGI)"


In [ ]:
from mlxtend.frequent_patterns import association_rules

**Yapılacaklar: Kural oluşturmak için metric olarak "confidence" i alarak minimum eşik değeri 0.6 olacak şekilde association_rules() fonksiyonunu kullanınız.**

Link: http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

In [ ]:
# Aprioride "support" hesabı yapabiliriz, "confidence ve diğerleri" için association rules kullanıyoruz.

association_rules(df1, metric = "confidence", min_threshold = 0.6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(COCK),(BISCUIT),0.15,0.35,0.10,0.666667,1.904762,0.0475,1.950
1,(BOURNVITA),(BREAD),0.20,0.65,0.15,0.750000,1.153846,0.0200,1.400
2,(JAM),(BREAD),0.10,0.65,0.10,1.000000,1.538462,0.0350,inf
3,(MAGGI),(BREAD),0.25,0.65,0.15,0.600000,0.923077,-0.0125,0.875
4,(MILK),(BREAD),0.25,0.65,0.20,0.800000,1.230769,0.0375,1.750
...,...,...,...,...,...,...,...,...,...
81,"(TEA, CORNFLAKES, COFFEE)",(MILK),0.05,0.25,0.05,1.000000,4.000000,0.0375,inf
82,"(COFFEE, MILK, CORNFLAKES)",(TEA),0.05,0.35,0.05,1.000000,2.857143,0.0325,inf
83,"(TEA, MILK)","(COFFEE, CORNFLAKES)",0.05,0.20,0.05,1.000000,5.000000,0.0400,inf
84,"(TEA, COFFEE)","(MILK, CORNFLAKES)",0.05,0.10,0.05,1.000000,10.000000,0.0450,inf


```
antecedent support: Birincinin tek başına görülme olasılığı,
consequent support: İkincinin tek başına görülme olasılığı,
support: İkisinin birlikte görülme olasılığı,
confidence: İlki satıldığında ikinci ürünün satılma olasılığı,
lift: İlki satıldığında ikinci ürünün satılma olasılığı şu kadar kat arttı yorumu
```

## Selecting and Filtering Results 

**Yapılacaklar: association_rules() sonucu oluşturulan kuralları "confidence" değeri 0.6'dan büyük eşit ve "support" değeri 0.2'den büyük eşit olan kuralları filtreleyiniz.**

In [ ]:
#association_rules() sonucu oluşturulan kuralları "confidence" değeri 0.6'dan büyük eşit ve 
# "support" değeri 0.2'den büyük eşit olan kuralları filtreleyiniz.

rules = association_rules(df1, metric = "confidence", min_threshold = 0.6)

rules[ (rules['confidence'] >= 0.6) & (rules['support'] >= 0.2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(MILK),(BREAD),0.25,0.65,0.2,0.800000,1.230769,0.0375,1.75
5,(SUGER),(BREAD),0.30,0.65,0.2,0.666667,1.025641,0.0050,1.05
8,(CORNFLAKES),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.80
9,(SUGER),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.80
11,(MAGGI),(TEA),0.25,0.35,0.2,0.800000,2.285714,0.1125,3.25


# Reporting

Analiz;
- Alışverişlerde MAGGI'ın tek başına görülme olasığı %25, TEA'in tek başına görülme olasılığı %35.
- 100 alışverişin 20'sinde mutlaka MAGGI ve TEA beraber satın alınıyor.
- MAGGI satıldığında TEA satılma olasılığı 0.800.. yani %80.
- MAGGI satılan satışlarda TEA satılma olasılığı 2.28 kat artmaktadır.


Aksiyon Fikri: 
- MAGGI alan biri %80 gibi yüksek bir ihtimalle TEA almaktadır ve TEA satışını 2.28 artırmaktadır. Bu iki ürün birbirinden uzak yerlerde konumlandırılarak müşterinin market içi dolaşması sağlanabilir, bu süreçte müşteri diğer ürünlere göz atabilir ve yahut satın alabilir. 
